In [12]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.options import Options
import pymongo
import time
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import requests
import unidecode
import urllib
from urllib.error import URLError, HTTPError
import sys
import pandas as pd

In [13]:
#connection to the database
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["facebook_scraping"]
#selecting the collections
mycol = mydb["facebook_posts"]

In [41]:
for i in mycol.find().limit(3):
    print(i)

{'_id': ObjectId('62b0ab16845bccadda3c9692'), 'username': 'nlf', 'date': 'date1', 'text': 'test', 'comments': ['c1', 'c2'], 'nb_comments': '2', 'nb_likes': '4', 'nb_shares': '1', 'image_links': ['link1', 'link2']}
{'_id': ObjectId('62b0b07a4db02595c453c9e3'), 'username': 'Lise Magnier', 'date': 'September 26, 2019', 'text': 'Le décès du Président Jacques Chirac nous bouleverse tous. Son incroyable capacité d’empathie, sa proximité naturelle avec celles et ceux qui croisaient sa route ont laissé dans la mémoire de chaque ville et chaque village de France une image ou l’un de ces mots dont il avait le secret.Son amour de la France, au service de laquelle il a consacré toute sa vie, nous restera à jamais en héritage.Merci Monsieur le Président pour ce que vous avez fait pour notre Pays.Toutes mes pensées vont à sa famille, à ses proches.', 'comments': ["C'est un peu une partie de la grandeur de la France qui disparaît.", 'Un grand homme...'], 'image_links': ['https://scontent.ftun18-1.fna

In [47]:
#connection to facebook using selenium
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

#setting chrome's browser settings
WINDOW_SIZE = "1366,768"
chrome_options = Options()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
driver = webdriver.Chrome(ChromeDriverManager(version="102.0.5005.61").install(),chrome_options=chrome_options)
driver.get("https://www.facebook.com/")
driver.maximize_window()

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\NGONGANG Loic F\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\NGONGA~1\AppData\Local\Temp/ipykernel_16792/4237761028.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager(version="102.0.5005.61").install(),chrome_options=chrome_options)
C:\Users\NGONGA~1\AppData\Local\Temp/ipykernel_16792/4237761028.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager(version="102.0.5005.61").install(),chrome_options=chrome_options)


In [48]:
#Entering facebook account credentials
try:       
    property_facts = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div    ")))
    email = driver.find_element_by_xpath("//input[@id='email']")    
    password = driver.find_element_by_xpath("//input[@id='pass']") 
    
    #enter your email
    email.send_keys("")
    #enter your password
    password.send_keys("")
    
    #connection button
    driver.find_element_by_xpath("//button[@class='_42ft _4jy0 _6lth _4jy6 _4jy1 selected _51sy']").click()
    
except Exception as e:
    print(e)

C:\Users\NGONGA~1\AppData\Local\Temp/ipykernel_16792/4168508885.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  email = driver.find_element_by_xpath("//input[@id='email']")
C:\Users\NGONGA~1\AppData\Local\Temp/ipykernel_16792/4168508885.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  password = driver.find_element_by_xpath("//input[@id='pass']")
C:\Users\NGONGA~1\AppData\Local\Temp/ipykernel_16792/4168508885.py:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//button[@class='_42ft _4jy0 _6lth _4jy6 _4jy1 selected _51sy']").click()


In [53]:
#set the subject for the search query
try:    
    property_facts = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div    ")))
    subject = "le décès du président Jacques Chirac"
    driver.get("https://www.facebook.com/"+subject)
except Exception as e:
    print(e)
url = driver.current_url.split('/')

In [ ]:
#actions functions

def getBack(browser):
    if not browser.current_url.endswith(url[len(url)-1]):
        print('redirected!!!')
        browser.back()
        print('got back!!!')
        
def openSeeMore(browser):
    readMore = browser.find_elements(By.XPATH, ".//div[@class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p']")
    if len(readMore)>0:
        action=ActionChains(driver)
        try:
            action.move_to_element(readMore[0]).click().perform()
        except:
            try:
                driver.execute_script("arguments[0].click();", readMore[0])
            except:
                print('seeMore issue')
        getBack(driver)
    else:
         pass
        
def openComment(browser):    
    comment = browser.find_elements(By.XPATH, ".//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain' and contains(text(),'Comments')]")
    if len(comment)>0:
        action=ActionChains(driver)
        try:
            action.move_to_element(comment[0]).click().perform()
        except:
            try:
                driver.execute_script("arguments[0].click();", comment[0])
            except:
                print('openComment issue')
        getBack(driver)
    else:
        pass
        
def openMoreComment(browser): 
    moreComment = browser.find_elements(By.XPATH, ".//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p m9osqain' and contains(text(),'more comments')]")
    while len(moreComment)>0:
        action=ActionChains(driver)
        try:
            action.move_to_element(moreComment[0]).click().perform()
        except:
            try:
                driver.execute_script("arguments[0].click();", comment[0])
            except:
                print('openComment issue')
        getBack(driver)
        time.sleep(2)
        moreComment = browser.find_elements(By.XPATH, ".//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p m9osqain' and contains(text(),'more comments')]")

def openMoreComment2(browser): 
    moreComment = browser.find_elements(By.XPATH, ".//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p m9osqain' and contains(text(),'previous comments')]")
    while len(moreComment)>0:
        action=ActionChains(driver)
        try:
            action.move_to_element(moreComment[0]).click().perform()
        except:
            try:
                driver.execute_script("arguments[0].click();", comment[0])
            except:
                print('openComment issue')
        getBack(driver)
        time.sleep(2)
        moreComment = browser.find_elements(By.XPATH, ".//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p m9osqain' and contains(text(),'previous comments')]")


In [44]:
#for many pages
#getting results
result = []
pages_list = ['lib.ennajeh.3','nlfthebiebs','Free-lance-102148821705399']

for page in pages_list:
    try:    
        driver.get("https://www.facebook.com/"+page)
    except Exception as e:
        print(e)
    url = driver.current_url.split('/')

   
    #load all the posts of the page till the end of the page
    property_facts = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div ")))
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(5)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True
    
    ##scrolling to the top of the page
    htmlelement= driver.find_element(by=By.TAG_NAME,value='html')
    htmlelement.send_keys(Keys.HOME)

    #get all the posts of the page
    posts = driver.find_elements(By.XPATH,"//div[@class='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0']")
    print(len(posts))
    for i in range(len(posts)):
        print(i)
        item = {}
        item['id_page'] = page
        p = posts[i]

        #open all see more... text
        openSeeMore(p)

        #open comments section
        openComment(p)
        time.sleep(1)

        #load more comments
        openMoreComment(p)
        time.sleep(1)

        #load previous comments
        openMoreComment2(p)
        time.sleep(1)

        posts = driver.find_elements(By.XPATH,"//div[@class='du4w35lb k4urcfbm l9j0dhe7 sjgh65i0']")
        #print(len(posts))
        p = posts[i]
        #extract html code of the post
        bsp =bs(p.get_attribute('innerHTML'),'html.parser')

        #author
        request = bsp.find('a',{'class':'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gpro0wi8 oo9gr5id lrazzd5p'})
        item['username'] = request.get_text()

        #post's date
        request = bsp.find('span',{'class':'tojvnm2t a6sixzi8 abs2jz4q a8s20v7p t1p8iaqh k5wvi7nf q3lfd5jv pk4s997a bipmatt0 cebpdrjk qowsmv63 owwhemhu dp1hu0rb dhp61c6y iyyx5f41'})
        item['date'] = request.get_text()

        #post's text
        text = ""
        request = bsp.find('span',{'class':'d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw oo9gr5id hzawbc8m'})
        if request:
            if len(request) != 0:
                text = request.get_text()
            else:
                text = 'none'
            item['text'] = text

        #comments
        comments = []
        request = bsp.findAll('div',{'class':'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql'})
        if len(request) == 0:
            comments.append('none')
        else:
            for c in request:
                comments.append(c.get_text())
        item['comments'] = comments

        #close comments section
        openComment(p)

        #image links
        image_links = ['none']
        links = bsp.findAll('img')
        for l in links:
            attr = l.get_attribute_list('class')
            if not str(attr).find('None')!=-1:
                link = ' '.join(str(item) for item in attr)
                if str(link).find('i09qtzwb n7fi1qx3 datstx6m pmk7jnqg j9ispegn kr520xx4 k4urcfbm') != -1:#posts's pictures
                    if 'none' in image_links:
                        image_links.remove('none')
                    image_links.append(l.get_attribute_list('src')[0])
                if str(link) == 'k4urcfbm bixrwtb6 datstx6m q9uorilb':#profile picture changed by the user
                    if 'none' in image_links:
                        image_links.remove('none')
                    image_links.append(l.get_attribute_list('src')[0])      

        item['image_links'] = image_links

        #number of post's likes
        likes = bsp.find_all('span',{'class':'pcp91wgn'})
        if len(likes)>0:
            nb_likes = likes[0].get_text()
            temp = nb_likes.split('\xa0')
            nb_likes = '.'.join(temp[0].split(','))
            if 'K' in temp[0]:
                nb_likes = float(nb_likes.split('K')[0])
                nb_likes *= 1000
            if 'M' in temp[0]:
                nb_likes = float(nb_likes.split('M')[0])
                nb_likes *= 1000000
            nb_likes = int(nb_likes)
        else:
            nb_likes = 0


        item['nb_likes'] = str(nb_likes)

        #number of post's comments and shares
        nb_cm = 0
        nb_sh = 0
        nb_comm_sh = bsp.find_all('span',{'class':'d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain'})
        for i in nb_comm_sh:
            text = i.get_text()
            if text.find('Comment')!=-1:
                nb_cm = text.split(' ')[0]
                temp = nb_cm.split('\xa0')
                nb_cm = '.'.join(temp[0].split(','))
                if 'K' in temp[0]:
                    nb_cm = float(nb_cm.split('K')[0])
                    nb_cm *= 1000
                if 'M' in temp[0]:
                    nb_cm = float(nb_cm.split('M')[0])
                    nb_cm *= 1000000
                nb_cm = int(nb_cm)

            if text.find('Share')!=-1:
                nb_sh = text.split(' ')[0]
                temp = nb_sh.split('\xa0')
                nb_sh = '.'.join(temp[0].split(','))
                if 'K' in temp[0]:
                    nb_sh = float(nb_sh.split('K')[0])
                    nb_sh *= 1000
                if 'M' in temp[0]:
                    nb_sh = float(nb_sh.split('M')[0])
                    nb_sh *= 1000000
                nb_sh = int(nb_sh)

        item['nb_comments'] = str(nb_cm)
        item['nb_shares'] = str(nb_sh)


        result.append(item)
        #insertion in the collection
        mycol.insert_one(item)


13
0
1
2
3
4
5
6
7
8
9
10
11
12
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1
0


In [45]:
result

[{'id_page': 'lib.ennajeh.3',
  'username': 'Lib Ennajeh',
  'date': 'June 19 at 7:46 PM',
  'text': 'ISIE - Tunis 1',
  'comments': ['none'],
  'image_links': ['none'],
  'nb_likes': '0',
  'nb_comments': '0',
  'nb_shares': '1'},
 {'id_page': 'lib.ennajeh.3',
  'username': 'Lib Ennajeh',
  'date': 'June 14 at 5:59 PM',
  'text': 'ساعدني الصفحة الرسمية',
  'comments': ['none'],
  'image_links': ['https://scontent.ftun7-1.fna.fbcdn.net/v/t39.30808-6/286928526_2331256073678861_4695792834611899821_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=DdWx0SZUNdwAX-NpeTk&_nc_ht=scontent.ftun7-1.fna&oh=00_AT-otECnxvBmETtQtKhPtOkDUUUJnpEwGdSth_c01DOUdA&oe=62C8EE0C',
   'https://scontent.ftun7-1.fna.fbcdn.net/v/t39.30808-6/285651933_2331256137012188_3871899275335179277_n.jpg?_nc_cat=103&ccb=1-7&_nc_sid=8bfeb9&_nc_ohc=whki0tRNcWwAX-wtKfC&_nc_ht=scontent.ftun7-1.fna&oh=00_AT86IwBgGvRdwNL_drWtDRkgjILCTWtq9VdmZvSsyx_kfw&oe=62C82348',
   'https://scontent.ftun7-1.fna.fbcdn.net/v/t39.30808-6/287225321_

In [46]:
pd.DataFrame(result)

,id_page,username,date,text,comments,image_links,nb_likes,nb_comments,nb_shares
0,lib.ennajeh.3,Lib Ennajeh,June 19 at 7:46 PM,ISIE - Tunis 1,[none],[none],0,0,1
1,lib.ennajeh.3,Lib Ennajeh,June 14 at 5:59 PM,ساعدني الصفحة الرسمية,[none],[https://scontent.ftun7-1.fna.fbcdn.net/v/t39....,11,1,1
2,lib.ennajeh.3,Lib Ennajeh,June 13 at 12:09 PM,RIADH MADHI - La Seule Page -,[none],[https://scontent.ftun7-1.fna.fbcdn.net/v/t39....,8,3,1
3,lib.ennajeh.3,Lib Ennajeh,May 17,قطاع التّعليم الأساسي,[none],[https://scontent.ftun7-1.fna.fbcdn.net/v/t39....,2,0,1
4,lib.ennajeh.3,Lib Ennajeh,March 28,NaN,[none],[none],13,6,0
5,lib.ennajeh.3,Lib Ennajeh,March 21,Education news-الاخبارية التربوية,[none],[https://scontent.ftun7-1.fna.fbcdn.net/v/t39....,0,0,0
6,lib.ennajeh.3,Lib Ennajeh,March 2,NaN,[none],[none],13,3,0
7,lib.ennajeh.3,Lib Ennajeh,February 26,بكري كان السارح سراحو الكلب نباح و المرا قايمة...,[none],[none],3,0,1
8,lib.ennajeh.3,Lib Ennajeh,February 26,NaN,[none],[none],7,0,0
9,lib.ennajeh.3,Lib Ennajeh,February 24,NaN,[none],[none],11,4,1
